In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']
AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"

In [3]:
def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('table').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'table': 'Table', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_tabellen'}).sort_index()
    return df

In [4]:
xl = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl['Objecten']
ignore_kolommen = df_table[df_table['Object'] == 'Ignore']['Tabellen'].values[0]
ignore_lst = ast.literal_eval(ignore_kolommen)

ignore_lst

['.*backup.*', '.*kopie.*']

In [5]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}},
            {"$group": { "_id": { 'project': "$project", 'table': "$table", 'teller': '$teller'}}},
            {'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$_id.teller"}}}]

df_brondata = AggregateData(pd.DataFrame(list(metaCollection.aggregate(grp_aggr))), 'Brondata')
df_brondata.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,AARDEWERK 1,"[DC97, DB34]",4618,2,Brondata
1,Aardewerk 1 backup,[DB34],4465,1,Brondata
2,VONDSTENLIJST,"[DC97, DC20, DC93, DC22, DC18, DC112, DB34, DC11]",3017,8,Brondata
3,DIAOPGRAVING,"[DC93, DC21, DC04, DC20, DB34, DC23, DC97, DC2...",1547,9,Brondata
4,Controle vondsten,[DB34],1466,1,Brondata


In [15]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}}
           ,{"$group": { "_id": { 'project': "$project", 'table': "$table"}, 'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}]

df_staging = AggregateData(pd.concat([pd.DataFrame(list(stagingOud.aggregate(grp_aggr))), pd.DataFrame(list(stagingNieuw.aggregate(grp_aggr)))]), 'Staging')
df_staging.head(5)
df_staging[df_staging.Table.str.contains('VONDST')]

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
2,VONDSTENLIJST,"[DB34, DC112, DC22, DC11, DC20, DC93, DC97, DC18]",3017,8,Staging
22,VONDSTINHD,"[DC179, DC154, DC24_STADSKANTOOR]",300,3,Staging
29,VONDST,"[DC179, DC24_STADSKANTOOR, DC154]",205,3,Staging
30,VAK_VONDST,[DC179],204,1,Staging
40,LIJST VONDSTEN URGENTE CONSERVERING,[DB34],129,1,Staging


In [12]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'table': "$brondata.table"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}]

df_singlestore = AggregateData(pd.DataFrame(list(analyseCol.aggregate(grp_aggr))), 'SingleStore')
df_singlestore.head(5)


,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
3,VONDSTENLIJST,"[DC18, DC22, DC112, DC97, DB34, DC93, DC20, DC11]",3017,8,SingleStore
20,VONDST,"[DC24_STADSKANTOOR, DC154, DC179]",205,3,SingleStore


In [10]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'table': "$brondata.table"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}]

df_singlestoreclean = AggregateData(pd.DataFrame(list(analyseColClean.aggregate(grp_aggr))), 'SingleStoreClean')
df_singlestoreclean.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
3,VONDSTENLIJST,"[DB34, DC112, DC22, DC11, DC20, DC93, DC97, DC18]",3017,8,SingleStoreClean
21,VONDST,"[DC179, DC24_STADSKANTOOR, DC154]",205,3,SingleStoreClean


In [9]:
lst_tables = ['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact', 'Def_Spoor', 'Def_Doos']
regexTable = re.compile(r'\'table\': \'(.*?)\'') # regex to replace Object
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getTable(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexTable.search(brondata).group(1)
    else: 
        return "" 
def getProject(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)
        
        df_out = pd.DataFrame()
        df_out['table'] = df.apply(lambda x: getTable(x['brondata']), axis=1)
        df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'table']).size().reset_index(name='teller')])
    
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)

ValueError: Wrong number of items passed 2, placement implies 1

## Analyse van alle gegevens

Hieronder volgt een overzicht van alle gegevens die in de vier fases beschikbaar zijn. Zo kunnen fouten in de conversie worden opgespoord. 

In [ ]:
def getOverview(OnWhat): 
    lst_countColumns = [OnWhat + '_brondata', OnWhat + '_staging', OnWhat + '_singlestore', OnWhat + '_singlestoreclean', OnWhat + '_doelsysteem']
    
    df = df_brondata[['Table', 'Projecten', OnWhat]].merge(df_staging[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_brondata", "_staging"))
    df = df.merge(df_singlestore[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_staging", "_singlestore"))
    df = df.merge(df_singlestoreclean[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_singlestore", "_singlestoreclean"))
    df = df.merge(df_doelsysteem[['Table', OnWhat]], on=['Table'], how='outer', suffixes=("_singlestoreclean", "_doelsysteem"))

    df = df.sort_values(OnWhat+ '_brondata', ascending=False) 
    df.rename(columns={'Aantal_Records': 'Aantal_Records_doelsysteem'}, inplace=True)
    df[lst_countColumns] = df[lst_countColumns].fillna(0)
    df[lst_countColumns] = df[lst_countColumns].astype(int, errors='ignore')
    df = df[df.Table.str.contains('vondst', case=False)]
    df = df.style.bar(subset=lst_countColumns, color='#5fba7d', vmax=6000)
    
    return df

getOverview('Aantal_Records')

## Detailanalyse Brondata naar Staging

In [ ]:
df = df_brondata.merge(df_staging, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_brondata", "_staging"))
df[df.Stage_brondata.isnull() | df.Stage_staging.isnull() ]

## Detailanalyse Staging naar SingleStore

In [ ]:
df = df_staging.merge(df_singlestore, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_staging", "_singlestore"))
df[df.Stage_singlestore.isnull() | df.Stage_staging.isnull() ]

## Detailanalyse SingleStore naar Doelsysteem

In [ ]:
df = df_singlestore.merge(df_doelsysteem, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
df[df.Stage_singlestore.isnull() | df.Stage_doelsysteem.isnull() ]

In [ ]:
# Experiments to be able to parse the mongo bson strings to either JSON or dict
# Not wirking :(((


startQ_regex = re.compile(r"({|\(|,\s*|:\s*)\'") # regex to replace first quote of words to double quaote
endQ_regex = re.compile(r"\'(:|\)|,|})") # regex to replace last quote of words to double quaote
regex = re.compile(r'ObjectId\((.*)\)') # regex to replace Object

# Does not work :(
def getFromJson(brondata):    
    if not brondata or brondata != "":
        brondata = brondata.replace('\"', '\'')
        brondata = startQ_regex.sub('\\1"', brondata)
        brondata = endQ_regex.sub('"\\1', brondata)
        brondata = regex.sub('\\1', brondata)
        brondata = brondata.replace('),', ',') # Quick fix because one ) could not be removed
        print(brondata)
        obj = json.loads(brondata)
        
        return obj
    else: 
        return {} 


#import re
#from bson.json_util import dumps, loads

#p = re.compile(r'blue (?P<animal>dog|cat)')
#p.sub(r'gray \g<animal>',s)
#re.sub(
#    pattern=r'ObjectId\((.*)\)', 
#    repl='\\1', 
#    string=str
#)

str = "{'_id': ObjectId('61e5f4a0ef919b0974d0395d'), 'LOKATIE': 'H:\\GEMEENTES\\PLAATS\\Opgravingen\\NAAM\\opgravingCODE.mdb', 'CODE': 'PN023', 'TOPONIEM': "Karitaat 'Molensloot", 'OPGRAVING': 'Vispaaiplaats Ruijven', 'CODENAAM': 'PN023', 'KAARTBLAD': '37E', 'XCOORD': 87523, 'YCOORD': 445002, 'JAAR': 2013, 'VONDSTENLIJST': 0, 'SPOREN': 0, 'DIAOPGRAVING': 0, 'DIAVOORWERP': 0, 'TEKENINGEN': 0, 'ROMEINS AARDEWERK': 0, 'AARDEWERK 1': 0, 'AARDEWERK 2': 0, 'KLEIPIJPEN': 0, 'TERRA COTTA': 0, 'GLAS': 0, 'BEEN': 0, 'BOT': 0, 'HOORN': 0, 'IVOOR': 0, 'HOUT': 0, 'METAAL': 0, 'MUNTEN EN PENNINGEN': 0, 'STEEN': 0, 'LEER': 0, 'TEXTIEL': 0, 'MODERN': 0, 'BIOLOGISCH': 0, 'SPECIAL': 0, 'IJZ': 0, 'ROM': 0, 'MIDDELEEUWS OF LATER': 0, 'VME': 0, 'LME': 0, 'LMEA': 0, 'LMEB': 0, 'NT': 0, 'WAARNEMING': 57944, 'table': 'OPGRAVINGEN', 'project': 'DELF-IT', 'bron': 'opgravingDELF-IT', 'loadtime': '2022-01-17T22:58:31+00:00'}"
#str = str.replace("\\", "/")
#str = str.replace("\'", "###")
#str = str.replace("\"", "\'")
#str = str.replace("###", "\"")


#str = re.sub(
#    pattern=r'ObjectId\((.*)\)', 
#    repl='\\1', 
#    string=str)

#json.loads(str)
#str = re.sub(
#    pattern=r"\'(.*?)\'(:|}|,)", 
#    repl='"\\1"\\2', 
#    string=str
#)
from ast import literal_eval
#d = eval(str)
#d['_id']

import yaml
yaml.load(str)

In [ ]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}}]

df_text = pd.DataFrame(list(metaCollection.aggregate(grp_aggr)))
text = ' '.join(list(df_text['name']))
text

In [ ]:
# Import package
import matplotlib.pyplot as plt
# Define a function to plot word cloud
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(40, 30))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off")

# Import package
from wordcloud import WordCloud, STOPWORDS
# Generate word cloud
wordcloud = WordCloud(width= 3000, height = 2000, collocations=False, stopwords = STOPWORDS).generate(text)
# Plot
plot_cloud(wordcloud)